In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Add the parent directory of 'src' to the Python path
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

# Longman phrase and anki deck generation
A more memorable way to learn the core vocabulary as defined in Longman Communications vocab lists, we take the vocab and use an LLM
to generate phrases using it.

## Longman 1000, 2000 and 3000 already provided
Enlish phrases for the longman vocab have already been created and can be found in the 'data' folder

In [32]:
from src.utils import get_longman_verb_vocab_dict
from src.phrase import generate_phrases_from_vocab_dict

file_path = '../data/longman-communication-3000.json'
vocab_dict = get_longman_verb_vocab_dict(file_path, "S3") #S1 = 1st 1000 words used in Speech, options are S1-3 and W1-3

In [ ]:
#uses LLM calls - it sometimes generates phrases terminated too early (e.g. Mind the pot on the), so advise you scan through and check

#english phrases only initially
longman_phrases = generate_phrases_from_vocab_dict(vocab_dict, max_iterations=15)

In [34]:
with open("../outputs/longman/longman_3000_phrases.txt", "w", encoding="utf-8") as f:
    for phrase in longman_phrases:
        f.write(phrase + "\n")

## Create an Anki deck from thoses phrases

Or, load one of the english Longman phrase lists in data/ already created and apply to your language

This function:
1. translates
2. generates audio using text to speech
3. packages up the text and audio into several anki decks (in batches), that can be imported into Anki.

The deck_name will is used to derive the deck_id and so despite there being several *.apkg files created, these will all merge successfully into the same deck


In [ ]:

from src.anki import create_anki_deck_from_english_phrase_list

_ = await create_anki_deck_from_english_phrase_list(longman_phrases[:3], deck_name="Longman 3000 - Swedish", anki_filename_prefix="longman_3000_swedish", batch_size=50)

In [42]:
from src.utils import create_image_generation_prompt


prompt = create_image_generation_prompt(longman_phrases[0])

In [ ]:
longman_phrases[0]

In [ ]:
print(prompt)

In [1]:
from diffusers import DiffusionPipeline
import torch
import os

# Check CUDA availability
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")
    


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

CUDA available: True
CUDA device: NVIDIA GeForce RTX 2070


In [8]:
import transformers

In [ ]:
transformers.util

In [3]:
os.getenv("TORCH_HOME")

In [6]:
torch.hub.set_dir("Y:/ai_caches/torch")

print(f"Hugging Face cache directory: {torch.hub.get_dir()}")

Hugging Face cache directory: Y:/ai_caches/torch


In [ ]:

pipe = DiffusionPipeline.from_pretrained("stabilityai/stable-diffusion-xl-base-1.0", torch_dtype=torch.float16, use_safetensors=True, variant="fp16")
pipe.to("cuda")


In [ ]:

# if using torch < 2.0
# pipe.enable_xformers_memory_efficient_attention()

prompt = "An exhausted engineer on a couch with a half-eaten pizza nearby. Next to the couch is a broken cooker with its parts scattered around"

images = pipe(prompt=prompt).images[0]
images.show()


In [ ]:
import torch
from diffusers import DiffusionPipeline, EulerDiscreteScheduler

# Use a smaller, faster model
model_id = "stabilityai/stable-diffusion-2-1-base"  # Smaller than XL

# Load the pipeline with optimizations
pipe = DiffusionPipeline.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    use_safetensors=True,
)

# Move to GPU and enable memory efficient attention
pipe = pipe.to("cuda")
pipe.enable_attention_slicing()

# Use the Euler scheduler, which is faster than the default
pipe.scheduler = EulerDiscreteScheduler.from_config(pipe.scheduler.config)

# Optional: Enable torch.compile() for potential speedup (requires PyTorch 2.0 or later)
# Uncomment the next line if you're using PyTorch 2.0+
# pipe.unet = torch.compile(pipe.unet, mode="reduce-overhead", fullgraph=True)


In [8]:

# Generate an image
prompt = """An exhausted engineer on a couch with a half-eaten pizza nearby. Next to the couch is a broken cooker with its parts scattered around. Hand-painted appearance, soft pastel colors, 
simple shapes, whimsical details. """
image = pipe(
    prompt,
    num_inference_steps=50,  # Reduced from default 50
    guidance_scale=7,        # Slightly reduced from default 7.5
    height=512,              # Reduced from default 1024 for SDXL
    width=512,               # Reduced from default 1024 for SDXL
).images[0]

# Save the image
image.show()


In [ ]:
import time
import torch
from diffusers import StableDiffusionPipeline
from PIL import Image

def benchmark_model(model_id, num_runs=5):
    pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
    pipe = pipe.to("cuda")
    pipe.enable_attention_slicing()

    prompt = "A bride on a balcony looking at a sunset."
    
    total_time = 0
    for run in range(num_runs):
        start_time = time.time()
        image = pipe(prompt, num_inference_steps=30, height=512, width=512).images[0]
        end_time = time.time()
        inference_time = end_time - start_time
        total_time += inference_time

        # Save the image
        image_filename = f"{model_id.split('/')[-1]}_{run+1}.png"
        image.save(image_filename)
        
        print(f"Run {run+1}:")
        print(f"  Model: {model_id}")
        print(f"  Inference time: {inference_time:.2f} seconds")
        print(f"  Image saved as: {image_filename}")
        print()

    avg_time = total_time / num_runs
    print(f"{model_id}: Average inference time: {avg_time:.2f} seconds")
    print("--------------------")

# List of models to benchmark
models = [
    "runwayml/stable-diffusion-v1-5",
    "stabilityai/stable-diffusion-2-1",
    "stabilityai/stable-diffusion-2-1-base",
    "stabilityai/stable-diffusion-xl-base-1.0",
]

for model in models:
    benchmark_model(model, num_runs=1)

print("Benchmark complete. Check the current directory for the generated images.")


Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

model.safetensors:  19%|#9        | 94.4M/492M [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   3%|2         | 94.4M/3.44G [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:  34%|###4      | 115M/335M [00:00<?, ?B/s]

model.safetensors:   8%|7         | 94.4M/1.22G [00:00<?, ?B/s]

y:\Python Scripts\audio-language-trainer\.venv\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\i5\.cache\huggingface\hub\models--runwayml--stable-diffusion-v1-5. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [ ]:

# Generate an image
prompt = "An exhausted engineer on a couch with a half-eaten pizza nearby. Next to the couch is a broken cooker with its parts scattered around"
image = pipe(prompt).images[0]
image.show()
